In [30]:
import numpy as np
import pandas as pd
import spacy
import string
import pickle as pkl

#load in the data
train_df = pd.read_csv('snli_train.tsv', sep="\t")
val_df = pd.read_csv('snli_val.tsv',sep="\t")

#get data & convert sentences to lists
train_sentence1=train_df['sentence1'].values.tolist()
train_sentence2=train_df['sentence2'].values.tolist()
val_sentence1=val_df['sentence1'].values.tolist()
val_sentence2=val_df['sentence2'].values.tolist()

In [31]:
# Load English tokenizer, tagger, parser, NER and word vectors
tokenizer = spacy.load('en_core_web_sm')
punctuations = string.punctuation

# lowercase and remove punctuation
def tokenize(sent):
  tokens = tokenizer(sent)
  return [token.text.lower() for token in tokens if (token.text not in punctuations)]

In [24]:
# This is the code cell that tokenizes train/val/test datasets
def tokenize_dataset(dataset):
    token_dataset = []
    # we are keeping track of all tokens in dataset 
    # in order to create vocabulary later
    all_tokens = []
    
    for sample in dataset:
        tokens = tokenize(sample)
        token_dataset.append(tokens)
        all_tokens += tokens

    return token_dataset, all_tokens

Tokenizing train data
Tokenizing val data


In [ ]:
#run this cell only once, otherwise leverage the existing saved files
#train set tokens
print ("Tokenizing train data")
train_sentence1_tokens, train_all_sentence1_tokens = tokenize_dataset(train_sentence1)
train_sentence2_tokens, train_all_sentence2_tokens = tokenize_dataset(train_sentence2)
pkl.dump(train_sentence1_tokens, open("train_sentence1_tokens.p", "wb"))
pkl.dump(train_sentence2_tokens, open("train_sentence2_tokens.p", "wb"))
pkl.dump(train_all_sentence1_tokens, open("train_all_sentence1_tokens.p", "wb"))
pkl.dump(train_all_sentence2_tokens, open("train_all_sentence2_tokens.p", "wb"))

#val set tokens
print ("Tokenizing val data")
val_sentence1_tokens, _ = tokenize_dataset(val_sentence1)
val_sentence2_tokens, _ = tokenize_dataset(val_sentence2)
pkl.dump(val_sentence1_tokens, open("val_sentence1_tokens.p", "wb"))
pkl.dump(val_sentence2_tokens, open("val_sentence2_tokens.p", "wb"))

In [33]:
#If you have previously run the previous cell, run this cell instead to load preprocessed datasets
train_sentence1_tokens = pkl.load(open("train_sentence1_tokens.p", "rb"))
train_sentence2_tokens = pkl.load(open("train_sentence2_tokens.p", "rb"))
train_all_sentence1_tokens = pkl.load(open("train_all_sentence1_tokens.p", "rb"))
train_all_sentence2_tokens = pkl.load(open("train_all_sentence2_tokens.p", "rb"))
val_sentence1_tokens = pkl.load(open("val_sentence1_tokens.p", "rb"))
val_sentence2_tokens = pkl.load(open("val_sentence2_tokens.p", "rb"))

In [34]:
#print information about the token datasets
# double checking
print ("Train sentence1 dataset size is {}".format(len(train_sentence1_tokens)))
print ("Train sentence2 dataset size is {}".format(len(train_sentence2_tokens)))
print ("Val sentence1 dataset size is {}".format(len(val_sentence1_tokens)))
print ("Val sentence2 dataset size is {}".format(len(val_sentence2_tokens)))

print ("Total number of tokens in sentence1 train dataset is {}".format(len(train_all_sentence1_tokens)))
print ("Total number of tokens in sentence2 train dataset is {}".format(len(train_all_sentence2_tokens)))
print ("Total number of *unique* tokens in sentence1 train dataset is {}".format(len(set(train_all_sentence1_tokens))))
print ("Total number of *unique* tokens in sentence2 train dataset is {}".format(len(set(train_all_sentence2_tokens))))


Train sentence1 dataset size is 100000
Train sentence2 dataset size is 100000
Val sentence1 dataset size is 1000
Val sentence2 dataset size is 1000
Total number of tokens in sentence1 train dataset is 1294135
Total number of tokens in sentence2 train dataset is 743372
Total number of *unique* tokens in sentence1 train dataset is 14131
Total number of *unique* tokens in sentence2 train dataset is 15225


In [37]:
#build vocabularies for sentence1 and sentence2
from collections import Counter

max_vocab_size = 10000
# save index 0 for unk and 1 for pad
PAD_IDX = 0
UNK_IDX = 1

def build_vocab(all_tokens):
    # Returns:
    # id2token: list of tokens, where id2token[i] returns token that corresponds to token i
    # token2id: dictionary where keys represent tokens and corresponding values represent indices
    token_counter = Counter(all_tokens)
    vocab, count = zip(*token_counter.most_common(max_vocab_size))
    id2token = list(vocab)
    token2id = dict(zip(vocab, range(2,2+len(vocab)))) 
    id2token = ['<pad>', '<unk>'] + id2token
    token2id['<pad>'] = PAD_IDX 
    token2id['<unk>'] = UNK_IDX
    return token2id, id2token

token2id_sentence1, id2token_sentence1 = build_vocab(train_all_sentence1_tokens)
token2id_sentence2, id2token_sentence2 = build_vocab(train_all_sentence2_tokens)

In [42]:
# Lets check the dictionary by loading random token from it
import random 

random_token_id = random.randint(0, len(id2token_sentence1)-1)
random_token = id2token_sentence1[random_token_id]

print ("Token id {} ; token {}".format(random_token_id, id2token_sentence1[random_token_id]))
print ("Token {}; token id {}".format(random_token, token2id_sentence1[random_token]))

Token id 584 ; token various
Token various; token id 584


In [45]:
# convert token to id in the dataset.  After running this cell we will have converted the word tokens to indices
def token2index_dataset(tokens_data,token2id):
    indices_data = []
    for tokens in tokens_data:
        index_list = [token2id[token] if token in token2id else UNK_IDX for token in tokens]
        indices_data.append(index_list)
    return indices_data

train_sentence1_data_indices = token2index_dataset(train_sentence1_tokens,token2id_sentence1)
train_sentence2_data_indices = token2index_dataset(train_sentence2_tokens,token2id_sentence2)
val_sentence1_data_indices = token2index_dataset(val_sentence1_tokens,token2id_sentence1)
val_sentence2_data_indices = token2index_dataset(val_sentence2_tokens,token2id_sentence2)

# double checking
print ("Train sentence1 dataset size is {}".format(len(train_sentence1_data_indices)))
print ("Train sentence2 dataset size is {}".format(len(train_sentence2_data_indices)))
print ("Val sentence1 dataset size is {}".format(len(val_sentence1_data_indices)))
print ("Val sentence2 dataset size is {}".format(len(val_sentence2_data_indices)))

Train sentence1 dataset size is 100000
Train sentence2 dataset size is 100000
Val sentence1 dataset size is 1000
Val sentence2 dataset size is 1000


In [54]:
#visualize a random sentence1 training example
rand_training_example = random.randint(0, len(train_sentence1) - 1)
print (train_sentence1_tokens[rand_training_example])
print(train_sentence1_data_indices[rand_training_example])

['africans', 'of', 'all', 'ages', 'singing', 'their', 'hearts', 'out', 'surrounded', 'by', 'mother', 'nature']
[5218, 9, 199, 1635, 292, 62, 3981, 71, 295, 42, 554, 2126]
